# Covariate Forecasting
In this notebook we will explore utilizing mobility data to effectively forecast COVID-19 spread.

In [0]:
!pip install git+https://github.com/CoronaWhy/task-ts
!pip install loguru
!pip install wandb
#!pip install git+https://github.com/CoronaWhy/task-geo.git

In [0]:
from corona_ts.data_utils.data_crawler import load_data
import pandas as pd
!wandb login

In [0]:
!mkdir /usr/local/lib/python3.6/dist-packages/data

In [33]:
df = load_data()
df['month'] = df['date'].map(lambda x: x.month)
df['weekday'] = df['date'].map(lambda x: x.weekday())

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2882: DtypeWarning: Columns (2) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
/usr/local/lib/python3.6/dist-packages/corona_ts/data_utils/data_crawler.py:99: DtypeWarning: Columns (3) have mixed types.Specify dtype option on import or set low_memory=False.
  mobility_df = fetch_mobility_data()


In [0]:
def format_corona_data(region_df:pd.DataFrame, region_name:str):
  """
  Format data for a specific region into 
  a format that can be used with flow forecast. 
  """
  if region_name == 'county':
    region_name = region_df['full_county'].iloc[0]
  else:
    region_name = region_df['state'].iloc[0]
  #else:
    #region_name = region_df['country'].iloc[0]
  print(region_name)
  region_df['datetime'] = region_df['date']
  region_df['precip'] = 0
  region_df['temp'] = 0
  region_df = region_df.fillna(0)
  region_df['new_cases'] = region_df['cases'].diff()
  region_df.iloc[0]['new_cases'] = 0
  region_df= region_df.fillna(0)
  region_df.to_csv(region_name+".csv")
  return region_df, len(region_df), region_name+".csv"

def loop_through_geo_codes(df, column='full_county'):
  df_county_list = []
  df['full_county'] = df['region'] + "_" + df['sub_region'] 
  for code in df['full_county'].unique():
    mask = df['full_county'] == code
    df_code = df[mask]
    ts_count = len(df_code)
    if ts_count > 60:
      df_county_list.append(df_code)
  return df_county_list

### Exploring Mobility Data for NYC

In [22]:
df_nyc = df[df['sub_region']=='New York County']
df_nyc.head()

,level,country,region,sub_region,date,lat,long,cases,deaths,recovered,active,tested,hospitalized,discharged,mobility_retail_recreation,mobility_grocery_pharmacy,mobility_parks,mobility_transit_stations,mobility_workplaces,mobility_residential,full_county
1593,sub_region,United States,New York,New York County,2020-03-20,40.781,-73.977,1544,0,0,0,5812,0,0,-71.0,-21.0,-49.0,-64.0,-63.0,26.0,New York_New York County
5488,sub_region,United States,New York,New York County,2020-04-08,40.781,-73.977,12106,0,0,0,29083,0,0,-87.0,-49.0,-65.0,-81.0,-79.0,32.0,New York_New York County
6606,sub_region,United States,New York,New York County,2020-04-28,40.781,-73.977,21604,0,0,0,64303,0,0,-83.0,-41.0,-42.0,-78.0,-75.0,23.0,New York_New York County
10883,sub_region,United States,New York,New York County,2020-04-30,40.781,-73.977,22175,0,0,0,68807,0,0,-85.0,-49.0,-71.0,-80.0,-77.0,32.0,New York_New York County
14732,sub_region,United States,New York,New York County,2020-04-14,40.781,-73.977,16617,0,0,0,40517,0,0,-85.0,-45.0,-60.0,-79.0,-77.0,31.0,New York_New York County


In [0]:
df

### Running the forecast model
In this next section when will forecast with mobility data. This will require the

##

In [17]:
import os
import pandas as pd
from google.colab import auth
from datetime import datetime
auth.authenticate_user()
!gcloud source repos clone github_aistream-peelout_flow-forecast --project=gmap-997
os.chdir('/content/github_aistream-peelout_flow-forecast')
!git checkout -t origin/covid_fixes
!python setup.py develop
!pip install -r requirements.txt
!mkdir data
from flood_forecast.trainer import train_function
!pip install git+https://github.com/CoronaWhy/task-geo.git

DEBUG:google.auth.transport.requests:Making request: POST https://oauth2.googleapis.com/token


$ git clone https://github.com/AIStream-Peelout/flow-forecast
Cloning into '/content/github_aistream-peelout_flow-forecast'...
remote: Total 3781 (delta 2442), reused 3781 (delta 2442)
Receiving objects: 100% (3781/3781), 2.68 MiB | 12.04 MiB/s, done.
Resolving deltas: 100% (2442/2442), done.
Project [gmap-997] repository [github_aistream-peelout_flow-forecast] was cloned to [/content/github_aistream-peelout_flow-forecast].
Branch 'covid_fixes' set up to track remote branch 'covid_fixes' from 'origin'.
Switched to a new branch 'covid_fixes'
/usr/local/lib/python3.6/dist-packages/setuptools/dist.py:454: UserWarning: Normalizing '0.01dev' to '0.1.dev0'
  warnings.warn(tmpl.format(**locals()))
running develop
running egg_info
creating flood_forecast.egg-info
writing flood_forecast.egg-info/PKG-INFO
writing dependency_links to flood_forecast.egg-info/dependency_links.txt
writing requirements to flood_forecast.egg-info/requires.txt
writing top-level names to flood_forecast.egg-info/top_leve

In [0]:
def make_config_file(file_path, df_len, weight_path=None):
  run = wandb.init(project="covid-forecast" )
  wandb_config = wandb.config
  train_number = df_len * .7
  validation_number = df_len *.9
  config_default={                 
    "model_name": "MultiAttnHeadSimple",
    "model_type": "PyTorch",
    "model_params": {
      "number_time_series":9,
      "seq_len":wandb_config["forecast_history"], 
      "output_seq_len":wandb_config["out_seq_length"],
      "forecast_length":wandb_config["out_seq_length"]
     },
     "weight_path_add":{
         "excluded_layers":["last_layer.weight", "last_layer.bias"]
     },
    "dataset_params":
    {  "class": "default",
       "training_path": file_path,
       "validation_path": file_path,
       "test_path": file_path,
       "batch_size":wandb_config["batch_size"],
       "forecast_history":wandb_config["forecast_history"],
       "forecast_length":wandb_config["out_seq_length"],
       "train_end": int(train_number),
       "valid_start":int(train_number+1),
       "valid_end": int(validation_number),
       "target_col": ["new_cases"],
       "relevant_cols": ["new_cases", "month", "weekday", "mobility_retail_recreation",	"mobility_grocery_pharmacy",	"mobility_parks",	"mobility_transit_stations",	"mobility_workplaces",	"mobility_residential"],
       "scaler": "StandardScaler", 
       "interpolate": False
    },
    "training_params":
    {
       "criterion":"MSE",
       "optimizer": "Adam",
       "optim_params":
       {

       },
       "lr": wandb_config["lr"],
       "epochs": 10,
       "batch_size":wandb_config["batch_size"]
    
    },
    "GCS": False,
    
    "sweep":True,
    "wandb":False,
    "forward_params":{},
   "metrics":["MSE"],
   "inference_params":
   {     
         "datetime_start":"2020-04-21",
          "hours_to_forecast":10, 
          "test_csv_path":file_path,
          "decoder_params":{
              "decoder_function": "simple_decode", 
            "unsqueeze_dim": 1
          },
          "dataset_params":{
             "file_path": file_path,
             "forecast_history":wandb_config["forecast_history"],
             "forecast_length":wandb_config["out_seq_length"],
             "relevant_cols": ["new_cases", "month", "weekday", "mobility_retail_recreation",	"mobility_grocery_pharmacy",	"mobility_parks",	"mobility_transit_stations",	"mobility_workplaces",	"mobility_residential"],
             "target_col": ["new_cases"],
             "scaling": "StandardScaler",
             "interpolate_param": False
          }
      }
  }
  if weight_path: 
    config_default["weight_path"] = weight_path
  wandb.config.update(config_default)
  return config_default

sweep_config = {
  "name": "Default sweep",
  "method": "grid",
  "parameters": {
        "batch_size": {
            "values": [2, 3]
        },
        "lr":{
            "values":[0.001, 0.01]
        },
        "forecast_history":{
            "values":[1, 2, 3]
        },
        "out_seq_length":{
            "values":[1, 2, 3]
        }
    }
}
import wandb
def sweep_all_geo(df_list, region_level, start_index=0, end_index=38):
  for array_index in range(start_index, end_index):
    region_df, full_len, file_path = format_corona_data(df_list[array_index], region_level)
    sweep_id = wandb.sweep(sweep_config, project="covid-forecast")
    wandb.agent(sweep_id, lambda:train_function("PyTorch", make_config_file(file_path, full_len)))
df_list = loop_through_geo_codes(df)

In [0]:
sweep_all_geo(df_list, 'county', 0, len(df_list))

INFO:wandb.wandb_agent:Cleaning up finished run: xdqfhmdo
INFO:wandb.wandb_agent:Agent received command: run
INFO:wandb.wandb_agent:Agent starting run with config:
	batch_size: 3
	forecast_history: 2
	lr: 0.01
	out_seq_length: 2
DEBUG:wandb.wandb_config:wandb dir not provided, skipping defaults


wandb: Agent Starting Run: ijo68fxx with config:
	batch_size: 3
	forecast_history: 2
	lr: 0.01
	out_seq_length: 2
wandb: Agent Started Run: ijo68fxx


DEBUG:wandb.wandb_config:no defaults not found in config-defaults.yaml


DEBUG:git.cmd:Popen(['git', 'cat-file', '--batch-check'], cwd=/content/github_aistream-peelout_flow-forecast, universal_newlines=False, shell=None, istream=<valid stream>)
DEBUG:wandb.meta:probe for git information
DEBUG:git.cmd:Popen(['git', 'rev-parse', '--show-toplevel'], cwd=/content/github_aistream-peelout_flow-forecast, universal_newlines=False, shell=None, istream=None)
DEBUG:wandb.run_manager:Initialized sync for covid-forecast/ijo68fxx
INFO:wandb.run_manager:system metrics and metadata threads started
INFO:wandb.run_manager:checking resume status, waiting at most 10 seconds
INFO:wandb.run_manager:resuming run from id: UnVuOnYxOmlqbzY4Znh4OmNvdmlkLWZvcmVjYXN0Omlnb2RmcmllZA==
INFO:wandb.run_manager:upserting run before process can begin, waiting at most 10 seconds
INFO:wandb.run_manager:saving pip packages
INFO:wandb.run_manager:initializing streaming files api
INFO:wandb.run_manager:unblocking file change observer, beginning sync with W&B servers


interpolate should be below
Now loading and scaling California_Tulare County.csv
interpolate should be below
Now loading and scaling California_Tulare County.csv
interpolate should be below
Now loading and scaling California_Tulare County.csv
Using Wandb config:
wandb_version: 1

GCS:
  desc: null
  value: false
_wandb:
  desc: null
  value:
    cli_version: 0.8.36
    framework: torch
    is_jupyter_run: true
    is_kaggle_kernel: false
    python_version: 3.6.9
batch_size:
  desc: null
  value: 3
dataset_params:
  desc: null
  value:
    batch_size: 3
    class: default
    forecast_history: 2
    forecast_length: 2
    interpolate: false
    relevant_cols:
    - new_cases
    - month
    - weekday
    - mobility_retail_recreation
    - mobility_grocery_pharmacy
    - mobility_parks
    - mobility_transit_stations
    - mobility_workplaces
    - mobility_residential
    scaler: StandardScaler
    target_col:
    - new_cases
    test_path: California_Tulare County.csv
    train_end: 4

...

/content/github_aistream-peelout_flow-forecast/flood_forecast/evaluator.py:67: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['preds'][history_length:] = end_tensor_list
/content/github_aistream-peelout_flow-forecast/flood_forecast/trainer.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_acc = evaluate_model(trained_model, model_type, params["dataset_params"]["target_col"], params["metrics"], params["inference_params"], {})


Current historical dataframe
    Unnamed: 0       level  ... original_index       preds
3         8395  sub_region  ...              3    0.000000
4        11486  sub_region  ...              4    0.000000
5        15786  sub_region  ...              5    0.000000
6        16031  sub_region  ...              6  171.391632
7        22018  sub_region  ...              7  153.682556
8        22848  sub_region  ...              8  178.928558
9        24693  sub_region  ...              9  153.227966
10       26403  sub_region  ...             10  116.396614
11       28181  sub_region  ...             11   93.671143
12       28351  sub_region  ...             12  198.450790
13       29694  sub_region  ...             13  188.400162
14       29853  sub_region  ...             14  172.794785
15       31986  sub_region  ...             15  168.083267

[13 rows x 30 columns]


INFO:wandb.run_manager:file/dir modified: /content/github_aistream-peelout_flow-forecast/wandb/run-20200520_050601-h8gx89bx/config.yaml
INFO:wandb.run_manager:file/dir modified: /content/github_aistream-peelout_flow-forecast/wandb/run-20200520_050601-h8gx89bx/wandb-summary.json
INFO:wandb.run_manager:file/dir modified: /content/github_aistream-peelout_flow-forecast/wandb/run-20200520_050601-h8gx89bx/wandb-history.jsonl
DEBUG:matplotlib.font_manager:findfont: Matching sans\-serif:style=normal:variant=normal:weight=normal:stretch=normal:size=10.0.
DEBUG:matplotlib.font_manager:findfont: score(<Font 'STIXGeneral' (STIXGeneralBolIta.ttf) italic normal 700 normal>) = 11.335
DEBUG:matplotlib.font_manager:findfont: score(<Font 'DejaVu Sans Display' (DejaVuSansDisplay.ttf) normal normal 400 normal>) = 10.05
DEBUG:matplotlib.font_manager:findfont: score(<Font 'STIXSizeTwoSym' (STIXSizTwoSymReg.ttf) normal normal regular normal>) = 10.05
DEBUG:matplotlib.font_manager:findfont: score(<Font 'DejaV

wandb: Agent Finished Run: h8gx89bx 



INFO:wandb.run_manager:shutting down system stats and metadata service
INFO:wandb.wandb_agent:Running runs: ['h8gx89bx']
INFO:wandb.run_manager:file/dir modified: /content/github_aistream-peelout_flow-forecast/wandb/run-20200520_050601-h8gx89bx/wandb-events.jsonl
INFO:wandb.run_manager:file/dir modified: /content/github_aistream-peelout_flow-forecast/wandb/run-20200520_050601-h8gx89bx/wandb-summary.json
INFO:wandb.run_manager:file/dir modified: /content/github_aistream-peelout_flow-forecast/wandb/run-20200520_050601-h8gx89bx/wandb-history.jsonl
INFO:wandb.run_manager:file/dir created: /content/github_aistream-peelout_flow-forecast/wandb/run-20200520_050601-h8gx89bx/media/plotly/test_plot_all_21_38bf9de5.plotly.json
INFO:wandb.run_manager:file/dir modified: /content/github_aistream-peelout_flow-forecast/wandb/run-20200520_050601-h8gx89bx/media/plotly
INFO:wandb.run_manager:stopping streaming files and file change observer
